In [16]:
%load_ext nb_black
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
from scipy.stats import chi2_contingency
from scipy import stats

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [17]:
# x=pd.read_csv('../_xlable4_withNa_AllColumns.txt')
# y=pd.read_csv('../_targelable4_withNa_AllColumns.txt')

# y=y[['Unnamed: 0','S000501', ]]

<IPython.core.display.Javascript object>

In [25]:
# data= pd.merge(x,y, how='inner',left_on='Unnamed: 0', right_on='Unnamed: 0')
data = pd.read_csv("../../sep19SexAndAgeAddedFINAL DATASET_ver2.txt", sep=",")
data["max"] = np.min(data[["FIELD_6", "FIELD_7"]], axis=1)
data = data.drop(columns=["FIELD_6", "FIELD_7"])

<IPython.core.display.Javascript object>

In [26]:
# change field 38 to 3 group
condtion = [
    data.FIELD_38 == 0,
    (data.FIELD_38 > 0) & (data.FIELD_38 < 4),
    data.FIELD_38 >= 4,
]
choice = [0, 1, 2]
data["FIELD_38_C"] = np.select(condtion, choice, default=np.nan)

<IPython.core.display.Javascript object>

In [27]:
# add Class to the dataset
_class = []
for i in range(data.shape[0]):
    if (data.S000501[i] >= 140) | (data.S000502[i] >= 90):
        _class.append(2)
    elif (data.S000501[i] < 120) & (data.S000502[i] < 80):
        _class.append(0)
    else:
        _class.append(1)
data["CLASS"] = _class

<IPython.core.display.Javascript object>

In [28]:
# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[
    data.FIELD_15 != 1
]  # exclude people who are diagnosed for (high blood pressure)
data = data[
    data.FIELD_22 != 1
]  # exclude people who are on medication for high blood pressure

data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
data = data[
    data.FIELD_24 != 1
]  # exclude people who are on medication for hyperlipidemia
print(data.shape)

(466680, 407)


<IPython.core.display.Javascript object>

In [29]:
# split column types to categorical and numerical
numerical_cols = list(data.columns[~data.columns.str.startswith("FIELD")])
categorical_cols = list(data.columns[data.columns.str.startswith("FIELD")])
categorical_cols.append("CLASS")
numerical_cols.remove("SEX")
categorical_cols.append("SEX")

<IPython.core.display.Javascript object>

Calculate correlation Value - for numerical values
========

In [31]:
corr = data[numerical_cols].corr()
corr = corr.S000501
corrvalPD = pd.DataFrame()
corrvalPD["Col"] = corr.index
corrvalPD["val"] = abs(corr.values)

corrvalPD = corrvalPD.sort_values(by="val", ascending=False)
corrvalPD.head()

,Col,val
63,L120600,1.0
178,L518900,1.0
59,L117700,1.0
148,L501600,1.0
213,S000501,1.0


<IPython.core.display.Javascript object>

In [32]:
# count of none NA values of feature set
mydataset = data.copy()[numerical_cols]
colCount = []
for i in mydataset.columns:
    colCount.append([i, mydataset[i].dropna().shape[0]])

colCountPD = pd.DataFrame(colCount, columns=["Col", "Count"])
colCountPD.head()

,Col,Count
0,Unnamed: 0,466680
1,COMPARE,466680
2,AGE,466679
3,B_DAY,466679
4,L100100,446887


<IPython.core.display.Javascript object>

In [33]:
mergedColCorrCount = pd.merge(
    corrvalPD, colCountPD, how="inner", left_on="Col", right_on="Col"
)
mergedColCorrCount = mergedColCorrCount[mergedColCorrCount.Count > 50000]
mergedColCorrCount = mergedColCorrCount.sort_values(by="val", ascending=False)
mergedColCorrCount.head(20)

,Col,val,Count
4,S000501,1.000000,466410
10,CLASS,0.827811,466680
13,S000502,0.772022,466410
21,S000300,0.432773,465393
26,L190300,0.354765,465316
28,L190400,0.354575,466197
29,L190500,0.348331,465172
34,L100700,0.324801,447592
40,L103300,0.288110,397549
44,L103000,0.275932,465103


<IPython.core.display.Javascript object>

In [40]:
print(list(mergedColCorrCount.head(20).Col))

['S000501', 'CLASS', 'S000502', 'S000300', 'L190300', 'L190400', 'L190500', 'L100700', 'L103300', 'L103000', 'L100800', 'L100500', 'S000100', 'L101700', 'max', 'L101300', 'L101600', 'L103100', 'S010101', 'S010102']


<IPython.core.display.Javascript object>

Anova test - For categorical values [Questionnaire answers]
=======

In [35]:
mydataset = data.copy()[categorical_cols]  # [np.insert(categorical_cols,0,'L100800')]

# mydataset=mydataset.drop(columns=['FIELD_1','FIELD_2','FIELD_8','FIELD_10','FIELD_11','FIELD_12','FIELD_39','FIELD_88',
#                            'FIELD_89','FIELD_109','FIELD_110','FIELD_111','FIELD_70','FIELD_82','FIELD_85','FIELD_91',
#                            'FIELD_103','FIELD_106'])

# mydataset=mydataset.drop(columns=['FIELD_1','FIELD_2','FIELD_87','FIELD_8','FIELD_10','FIELD_11','FIELD_12','FIELD_39',
#                                   'FIELD_88','FIELD_89', 'FIELD_108','FIELD_109','FIELD_110','FIELD_111','FIELD_82',
#                                  'FIELD_91','FIELD_103','FIELD_118','FIELD_119','FIELD_120','FIELD_121','FIELD_122',
#                                  'FIELD_123','FIELD_124','FIELD_125','FIELD_126','FIELD_127','FIELD_128','FIELD_129',
#                                  'FIELD_130','FIELD_131','FIELD_132','FIELD_133','FIELD_134','FIELD_135','FIELD_136',
#                                   'FIELD_137','FIELD_138','FIELD_139','FIELD_140'])


mydataset = mydataset.drop(
    columns=[
        "FIELD_1",
        "FIELD_2",
        "FIELD_87",
        "FIELD_8",
        "FIELD_10",
        "FIELD_11",
        "FIELD_12",
        "FIELD_39",
        "FIELD_88",
        "FIELD_89",
        "FIELD_108",
        "FIELD_109",
        "FIELD_110",
        "FIELD_111",
        "FIELD_82",
        "FIELD_91",
        "FIELD_103",
        "FIELD_118",
        "FIELD_119",
        "FIELD_120",
        "FIELD_121",
        "FIELD_122",
        "FIELD_123",
        "FIELD_124",
        "FIELD_125",
        "FIELD_126",
        "FIELD_127",
        "FIELD_128",
        "FIELD_129",
        "FIELD_130",
        "FIELD_131",
        "FIELD_132",
        "FIELD_133",
        "FIELD_134",
        "FIELD_135",
        "FIELD_136",
        "FIELD_137",
        "FIELD_138",
        "FIELD_139",
        "FIELD_140",
        "FIELD_64",
        "FIELD_65",
        "FIELD_66",
        "FIELD_67",
        "FIELD_68",
        "FIELD_69",
        "FIELD_70",
        "FIELD_72",
        "FIELD_73",
        "FIELD_74",
        "FIELD_75",
        "FIELD_76",
        "FIELD_77",
        "FIELD_80",
        "FIELD_81",
        "FIELD_84",
        "FIELD_85",
        "FIELD_90",
        "FIELD_93",
        "FIELD_94",
        "FIELD_95",
        "FIELD_96",
        "FIELD_97",
        "FIELD_98",
        "FIELD_101",
        "FIELD_102",
        "FIELD_105",
        "FIELD_106",
        "FIELD_112",
        "FIELD_113",
        "FIELD_114",
        "FIELD_115",
        "FIELD_116",
        "FIELD_117",
        "FIELD_118",
        "FIELD_119",
        "FIELD_9",
    ]
)


mydataset = mydataset.drop(mydataset[mydataset.FIELD_3 == "`"].index)
mydataset = mydataset.drop(mydataset[mydataset.FIELD_3 == "G"].index)


mydataset = mydataset.drop(mydataset[mydataset.FIELD_4 == "."].index)
mydataset = mydataset.drop(mydataset[mydataset.FIELD_5 == "."].index)

mydataset = mydataset.drop(mydataset[mydataset.FIELD_35 == "?"].index)
mydataset = mydataset.drop(mydataset[mydataset.FIELD_35 == "."].index)
mydataset = mydataset.drop(mydataset[mydataset.FIELD_36 == "."].index)
mydataset = mydataset.drop(mydataset[mydataset.FIELD_37 == "\\"].index)

# mydataset=mydataset.drop(mydataset[mydataset.FIELD_39 == '7+' ].index)
# mydataset=mydataset.drop(mydataset[mydataset.FIELD_39 == '5~7'].index)
# mydataset=mydataset.drop(mydataset[mydataset.FIELD_39 == '3-4'].index)


# mydataset=mydataset.drop(mydataset[mydataset.FIELD_95=='.'].index)

<IPython.core.display.Javascript object>

In [36]:
colslist = mydataset.columns
DiabeticColVals = data.S000501

cor = []
pval = []
count = []

for i in range(len(colslist)):
    print(i, colslist[i])

    temp = pd.DataFrame([])
    temp["a"] = DiabeticColVals
    temp["b"] = mydataset[colslist[i]].astype(float)
    temp = temp.dropna()
    count.append(temp.shape[0])

    uniquevalues = temp.b.unique()

    selectedGroupVals = []
    for k in range(len(uniquevalues)):
        selectedGroupVals.append(np.asarray(temp[temp["b"] == uniquevalues[k]].a))

    F, p = stats.f_oneway(*selectedGroupVals)
    cor.append(F)
    pval.append(p)

0 FIELD_3
1 FIELD_4
2 FIELD_5
3 FIELD_13
4 FIELD_14
5 FIELD_15
6 FIELD_16
7 FIELD_17
8 FIELD_18
9 FIELD_19
10 FIELD_20
11 FIELD_21
12 FIELD_22
13 FIELD_23
14 FIELD_24
15 FIELD_25
16 FIELD_26
17 FIELD_27
18 FIELD_28
19 FIELD_29
20 FIELD_30
21 FIELD_31
22 FIELD_32
23 FIELD_33
24 FIELD_34
25 FIELD_35
26 FIELD_36
27 FIELD_37
28 FIELD_38
29 FIELD_40
30 FIELD_41
31 FIELD_42
32 FIELD_43
33 FIELD_44
34 FIELD_45
35 FIELD_46
36 FIELD_47
37 FIELD_48
38 FIELD_49
39 FIELD_50
40 FIELD_51
41 FIELD_52
42 FIELD_53
43 FIELD_54
44 FIELD_55
45 FIELD_56
46 FIELD_57
47 FIELD_58
48 FIELD_59
49 FIELD_60
50 FIELD_61
51 FIELD_62
52 FIELD_63
53 FIELD_38_C
54 CLASS
55 SEX


<IPython.core.display.Javascript object>

In [37]:
FvalPvalCorr = pd.DataFrame()
FvalPvalCorr["Cols"] = colslist
FvalPvalCorr["F"] = cor
FvalPvalCorr["P"] = pval
FvalPvalCorr["Count"] = count
FvalPvalCorr = FvalPvalCorr.sort_values(by="F", ascending=False)

<IPython.core.display.Javascript object>

In [38]:
FvalPvalCorr[FvalPvalCorr.Count > 50000].head(10)

,Cols,F,P,Count
54,CLASS,514321.357187,0.000000e+00,466397
55,SEX,70935.966231,0.000000e+00,466396
53,FIELD_38_C,4777.768439,0.000000e+00,433775
23,FIELD_33,3303.562953,0.000000e+00,464179
28,FIELD_38,1410.009764,0.000000e+00,433775
29,FIELD_40,489.080744,0.000000e+00,433843
19,FIELD_29,347.550606,1.491927e-151,463541
30,FIELD_41,315.885983,0.000000e+00,433771
17,FIELD_27,308.139192,1.844285e-134,463031
31,FIELD_42,60.368834,3.642285e-99,433726


<IPython.core.display.Javascript object>

In [39]:
print(list(FvalPvalCorr[FvalPvalCorr.Count > 50000].Cols)[:10])

['CLASS', 'SEX', 'FIELD_38_C', 'FIELD_33', 'FIELD_38', 'FIELD_40', 'FIELD_29', 'FIELD_41', 'FIELD_27', 'FIELD_42']


<IPython.core.display.Javascript object>

========================= LASSO method
================

In [42]:
selectedcols = [
    "S000501",
    "S000300",
    "L190300",
    "L190400",
    "L190500",
    "L100700",
    "L103300",
    "L103000",
    "L100800",
    "L100500",
    "S000100",
    "L101700",
    "max",
    "L101300",
    "L101600",
    "L103100",
    "S010101",
    "S010102",
    "SEX",
    "FIELD_38_C",
    "FIELD_33",
    "FIELD_38",
    "FIELD_40",
    "FIELD_29",
    "FIELD_41",
    "FIELD_27",
    "FIELD_42",
    "AGE",
]
data2 = data[selectedcols].copy().dropna()
print(data2.shape)

(138973, 28)


<IPython.core.display.Javascript object>

In [43]:
x = data2[
    [
        "S000300",
        "L190300",
        "L190400",
        "L190500",
        "L100700",
        "L103300",
        "L103000",
        "L100800",
        "L100500",
        "S000100",
        "L101700",
        "max",
        "L101300",
        "L101600",
        "L103100",
        "S010101",
        "S010102",
        "SEX",
        "FIELD_38_C",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_29",
        "FIELD_41",
        "FIELD_27",
        "FIELD_42",
        "AGE",
    ]
]
y = data2.S000501

<IPython.core.display.Javascript object>

In [44]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=0.1)
clf.fit(x, y)

print(clf.coef_)

print(clf.intercept_)

# pd.DataFrame([x.columns.ravel(),clf.coef_.ravel()],columns=['Name','coeff'])

lassod = pd.DataFrame()
lassod["Name"] = x.columns
lassod["coeff"] = clf.coef_
lassod

lassod = lassod.drop(lassod[lassod.coeff == 0].index)
lassod.coeff = abs(lassod.coeff)
lassod = lassod.sort_values(by="coeff", ascending=False)
lassod

[ 1.05219685e+00  3.45859055e+00 -0.00000000e+00  4.72486324e-02
  1.35961911e-01  0.00000000e+00  1.20612434e-02  9.53064603e-02
 -0.00000000e+00  1.09822687e-01  1.29863646e-02  0.00000000e+00
 -6.25728674e-03  3.78656422e-02  3.51051407e-02  2.87361582e-01
  2.37663801e-01 -0.00000000e+00  0.00000000e+00 -8.86797724e-02
  3.90573381e-01  1.13315026e-03  1.21631704e+00  6.33787797e-02
  0.00000000e+00  5.45027643e-02  2.71148380e-01]
21.794080732411757


,Name,coeff
1,L190300,3.458591
22,FIELD_29,1.216317
0,S000300,1.052197
20,FIELD_38,0.390573
15,S010101,0.287362
26,AGE,0.271148
16,S010102,0.237664
4,L100700,0.135962
9,S000100,0.109823
7,L100800,0.095306


<IPython.core.display.Javascript object>

In [45]:
print(list(lassod.Name[:16]))

['L190300', 'FIELD_29', 'S000300', 'FIELD_38', 'S010101', 'AGE', 'S010102', 'L100700', 'S000100', 'L100800', 'FIELD_33', 'FIELD_41', 'FIELD_42', 'L190500', 'L101600', 'L103100']


<IPython.core.display.Javascript object>

=============================== SelectKBest method
========================

In [46]:
# from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest

bestfeatures = SelectKBest(score_func=f_regression, k=10)
fit = bestfeatures.fit(x, y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ["Specs", "Score"]  # naming the dataframe columns

# x=x[x.columns[:15]]
print(list(x.columns[:16]))

['S000300', 'L190300', 'L190400', 'L190500', 'L100700', 'L103300', 'L103000', 'L100800', 'L100500', 'S000100', 'L101700', 'max', 'L101300', 'L101600', 'L103100', 'S010101']


<IPython.core.display.Javascript object>

========================================= LinearSVC
===============

In [41]:
# from sklearn.svm import LinearSVC
# from sklearn.datasets import load_iris
# from sklearn.feature_selection import SelectFromModel

# lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(x, y)
# model = SelectFromModel(lsvc, prefit=True)
# X_new = model.transform(x)
# X_new.shape

(21538, 20)

In [44]:
# X_new

================================ SequentialFeatureSelector 
================================

In [33]:
# from sklearn.linear_model import LinearRegression
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
# from sklearn.ensemble import RandomForestRegressor

# lr = RandomForestRegressor()

# sfs = SFS(lr, 
#           k_features=13, 
#           forward=True, 
#           floating=False, 
#           scoring='neg_mean_squared_error',
#           cv=5)

# sfs = sfs.fit(x, y)
# fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')

# plt.title('Sequential Forward Selection (w. StdErr)')
# plt.grid()
# plt.show()

================================= feature_importances
=====

In [47]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

model = ExtraTreesClassifier()
model.fit(x, y)

print(
    model.feature_importances_
)  # use inbuilt class feature_importances of tree based classifiers

# plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(16).plot(kind="barh")
plt.show()

print(feat_importances.nlargest(16))

[5.35014340e-02 5.16540782e-02 4.92644788e-02 5.04610909e-02
 5.10542687e-02 5.03288661e-02 5.15652485e-02 5.13416461e-02
 4.14807325e-02 5.14080430e-02 4.93140187e-02 2.15942140e-03
 4.94892934e-02 5.07936834e-02 5.09291525e-02 4.53456677e-02
 4.46244731e-02 1.42100132e-05 1.25047621e-02 9.22547337e-03
 1.92048612e-02 2.79561738e-02 5.59657657e-03 3.16391696e-02
 4.43561904e-03 4.42859558e-02 5.04216015e-02]


<Figure size 640x480 with 1 Axes>

S000300    0.053501
L190300    0.051654
L103000    0.051565
S000100    0.051408
L100800    0.051342
L100700    0.051054
L103100    0.050929
L101600    0.050794
L190500    0.050461
AGE        0.050422
L103300    0.050329
L101300    0.049489
L101700    0.049314
L190400    0.049264
S010101    0.045346
S010102    0.044624
dtype: float64


<IPython.core.display.Javascript object>

============================= Backward Elimination
========

In [50]:
import statsmodels.regression.linear_model as sm

temp = x.copy()
temp["const"] = np.ones((x.shape[0], 1))
regressor_OLS = sm.OLS(endog=y, exog=temp).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                S000501   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     1795.
Date:                Thu, 17 Oct 2019   Prob (F-statistic):               0.00
Time:                        23:00:27   Log-Likelihood:            -5.2689e+05
No. Observations:              138973   AIC:                         1.054e+06
Df Residuals:                  138945   BIC:                         1.054e+06
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
S000300        1.0375      0.011     96.367      0.000       1.016       1.059
L190300        5.3876      0.141     38.082      0.000       5.110       5.665
L190400       -0.0294      0.082     -0.359      0.720      -0.190       0.131
L190500       -0.0741      0.036     -2.066      0.039      -0.144      -0.004
L100700        0.2528      0.036      7.067      0.000       0.183       0.323
L103300        0.1796      0.055      3.276      0.001       0.072       0.287
L103000        0.0105      0.001     15.130      0.000       0.009       0.012
L100800        0.0933      0.003     32.229      0.000       0.088       0.099
L100500       -1.3036      0.254     -5.139      0.000      -1.801      -0.806
S000100        0.1191      0.006     20.869      0.000       0.108       0.130
L101700        0.0143      0.002      8.631      0.000       0.011       0.018
max            0.3025      0.104      2.913      0.004       0.099       0.506
L101300       -0.0085      0.002     -3.742      0.000      -0.013      -0.004
L101600        0.0323      0.002     15.166      0.000       0.028       0.036
L103100        0.0419      0.003     13.775      0.000       0.036       0.048
S010101        0.2917      0.016     17.707      0.000       0.259       0.324
S010102        0.2399      0.017     14.488      0.000       0.207       0.272
SEX           -5.4098      4.056     -1.334      0.182     -13.360       2.541
FIELD_38_C    -0.4381      0.107     -4.111      0.000      -0.647      -0.229
FIELD_33      -0.7708      0.075    -10.283      0.000      -0.918      -0.624
FIELD_38       0.7509      0.054     13.868      0.000       0.645       0.857
FIELD_40       0.0469      0.024      1.920      0.055      -0.001       0.095
FIELD_29       1.8676      0.077     24.382      0.000       1.717       2.018
FIELD_41       0.0799      0.022      3.577      0.000       0.036       0.124
FIELD_27       0.2388      0.123      1.941      0.052      -0.002       0.480
FIELD_42       0.0640      0.014      4.728      0.000       0.037       0.091
AGE            0.2699      0.004     62.716      0.000       0.261       0.278
const         23.0656      4.239      5.442      0.000      14.758      31.373
==============================================================================
Omnibus:                     6015.771   Durbin-Watson:                   1.580
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9013.269
Skew:                           0.405   Prob(JB):                         0.00
Kurtosis:                       3.949   Cond. No.                     4.64e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.64e+04. This might i

<IPython.core.display.Javascript object>

In [49]:
temp = temp.drop(
    columns=[
        "L103000",
        "L103300",
        "L190300",
        "FIELD_40",
        "FIELD_27",
        "FIELD_29",
        "FIELD_21",
        "FIELD_5",
        "FIELD_14",
    ]
)
regressor_OLS = sm.OLS(endog=y, exog=temp).fit()
regressor_OLS.summary()

KeyError: "['FIELD_21' 'FIELD_5' 'FIELD_14'] not found in axis"

<IPython.core.display.Javascript object>

In [49]:
temp=temp.drop(columns=['FIELD_33'])
regressor_OLS = sm.OLS(endog = y, exog = temp).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              L100800_y   R-squared (uncentered):                   0.992
Model:                            OLS   Adj. R-squared (uncentered):              0.992
Method:                 Least Squares   F-statistic:                          1.851e+05
Date:                Wed, 09 Oct 2019   Prob (F-statistic):                        0.00
Time:                        21:54:41   Log-Likelihood:                         -77458.
No. Observations:               21538   AIC:                                  1.549e+05
Df Residuals:                   21524   BIC:                                  1.551e+05
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
L100800_x      0.4959      0.007     72.088      0.000       0.482       0.509
L104600        8.5403      0.181     47.069      0.000       8.185       8.896
S000300        0.0717      0.022      3.239      0.001       0.028       0.115
SEX           -1.8595      0.163    -11.392      0.000      -2.179      -1.540
S000501        0.0194      0.008      2.494      0.013       0.004       0.035
L101700        0.0166      0.002      7.972      0.000       0.013       0.021
L190400        1.7212      0.153     11.220      0.000       1.421       2.022
S000502       -0.0279      0.010     -2.702      0.007      -0.048      -0.008
L190500       -0.6045      0.054    -11.168      0.000      -0.711      -0.498
L101300        0.0124      0.004      3.303      0.001       0.005       0.020
AGE            0.0271      0.008      3.354      0.001       0.011       0.043
FIELD_38       0.2745      0.056      4.941      0.000       0.166       0.383
FIELD_31       0.5832      0.169      3.449      0.001       0.252       0.915
FIELD_41       0.1317      0.038      3.491      0.000       0.058       0.206
==============================================================================
Omnibus:                    12859.970   Durbin-Watson:                   2.086
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           743785.719
Skew:                           2.144   Prob(JB):                         0.00
Kurtosis:                      31.468   Cond. No.                         648.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
temp.columns

Index(['L100800_x', 'L104600', 'S000300', 'SEX', 'S000501', 'L101700',
       'L190400', 'S000502', 'L190500', 'L101300', 'AGE', 'FIELD_38',
       'FIELD_31', 'FIELD_41'],
      dtype='object')

======================== variance inflation factor 
=============

In [51]:
# this function computes vif value which help us to remove columns with redundant information ( to avoid Multicollinearity)
# Multicollinearity occurs when two or more predictors in the model are correlated and provide redundant information about the response.
# Multicollinearity was measured by variance inflation factors (VIF) and tolerance


def vifcal(inputdata, depcol):
    vifL5 = []
    import statsmodels.formula.api as sm

    xvars = inputdata.drop([depcol], axis=1)
    xvarnames = xvars.columns
    for i in range(0, xvarnames.shape[0]):
        _y = xvars[xvarnames[i]]
        _x = xvars[xvarnames.drop(xvarnames[i])]
        rsq = sm.ols(formula="_y~_x", data=xvars).fit().rsquared
        vif = round(1 / (1 - rsq), 2)
        print(i, ", ", xvarnames[i], " VIF = ", vif)
        if vif < 5:
            vifL5.append(xvarnames[i])
    return vifL5

<IPython.core.display.Javascript object>

In [53]:
newcols = vifcal(data2, "S000501")
print(newcols, len(newcols))

0 ,  S000300  VIF =  1.37
1 ,  L190300  VIF =  2.32
2 ,  L190400  VIF =  10.03
3 ,  L190500  VIF =  12.97
4 ,  L100700  VIF =  1.21
5 ,  L103300  VIF =  2.91
6 ,  L103000  VIF =  1.62
7 ,  L100800  VIF =  1.21
8 ,  L100500  VIF =  1.08
9 ,  S000100  VIF =  1.2
10 ,  L101700  VIF =  1.35
11 ,  max  VIF =  2.15
12 ,  L101300  VIF =  1.28
13 ,  L101600  VIF =  1.46
14 ,  L103100  VIF =  2.06
15 ,  S010101  VIF =  2.69
16 ,  S010102  VIF =  2.69
17 ,  SEX  VIF =  1.0
18 ,  FIELD_38_C  VIF =  4.08
19 ,  FIELD_33  VIF =  1.06
20 ,  FIELD_38  VIF =  4.1
21 ,  FIELD_40  VIF =  1.45
22 ,  FIELD_29  VIF =  1.03
23 ,  FIELD_41  VIF =  1.58
24 ,  FIELD_27  VIF =  1.04
25 ,  FIELD_42  VIF =  1.14
26 ,  AGE  VIF =  2.37
['S000300', 'L190300', 'L100700', 'L103300', 'L103000', 'L100800', 'L100500', 'S000100', 'L101700', 'max', 'L101300', 'L101600', 'L103100', 'S010101', 'S010102', 'SEX', 'FIELD_38_C', 'FIELD_33', 'FIELD_38', 'FIELD_40', 'FIELD_29', 'FIELD_41', 'FIELD_27', 'FIELD_42', 'AGE'] 25


<IPython.core.display.Javascript object>